# Detecting changes in distribution using evidently
---

## Dependencies

In [1]:
!pip install -qq evidently

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 118.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 13.0.0 which is incompatible.


In [2]:
!jupyter nbextension install --sys-prefix --symlink --overwrite --py evidently

Installing /usr/local/lib/python3.10/dist-packages/evidently/nbextension/static -> evidently
Symlinking: /usr/share/jupyter/nbextensions/evidently -> /usr/local/lib/python3.10/dist-packages/evidently/nbextension/static
- Validating: OK

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable evidently --py --sys-prefix
    


In [3]:
!jupyter nbextension enable evidently --py --sys-prefix

Enabling notebook extension evidently/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import evidently

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metric_preset.classification_performance import ClassificationPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset
from evidently.tests import *

import joblib

## Data

In [5]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/OCL/P7/data/'

X = pd.read_csv(path+'X_train_45.csv')

Mounted at /content/drive


In [6]:
X

,AMT_INCOME_TOTAL,AMT_CREDIT,EXT_SOURCE_2,repayment_time_months,number_of_previous_loans,previous_amount_credit,number_of_years_since_last_decision,prev_contrat_statut_canceled_unused,prev_contrat_statut_refused,prev_contrat_type_consumer_loans,...,MONTHS_BALANCE_x,CNT_INSTALMENT,MONTHS_BALANCE_y,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,FLAG_OWN_CAR_No,FLAG_OWN_REALTY_Yes,CNT_FAM_MEMBERS_two,SK_ID_CURR
0,0.145504,-0.470083,-1.305371,-0.695997,-0.923233,0.064303,-0.315545,-0.564996,-0.460039,-0.631356,...,1.063556,1.183715,0.238308,-0.226703,-0.176182,-0.155874,1,1,0,100002
1,0.422496,1.797594,0.575515,1.891279,-0.440682,1.911626,0.519649,-0.564996,-0.460039,-0.081366,...,-0.610476,-0.545931,0.238308,-0.226703,-0.176182,-0.155874,1,0,1,100003
2,-0.408479,-1.164516,0.228265,-0.178542,-0.923233,-0.897989,-0.315545,-0.564996,-0.460039,-0.631356,...,0.295554,-1.337388,0.238308,-0.226703,-0.176182,-0.155874,0,1,0,100004
3,-0.131488,-0.710209,0.723118,-1.342817,1.006971,0.746240,-1.289938,1.186929,0.097237,-0.081366,...,1.082431,-0.310272,1.056734,-0.614430,-0.964472,-0.740095,1,1,1,100006
4,-0.186886,-0.198029,-0.992378,0.209549,0.283144,-0.010866,0.415250,-0.564996,-0.460039,-0.081366,...,-0.107591,0.104724,0.238308,-0.226703,-0.176182,-0.155874,1,1,0,100007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280825,-0.039157,-0.858461,0.886398,-1.601544,-0.923233,-0.774794,-0.941940,-0.564996,-0.460039,-0.631356,...,1.311298,-0.823830,0.238308,-0.226703,-0.176182,-0.155874,1,0,0,456251
280826,-0.390013,-0.820492,-2.074674,0.080185,-0.923233,-0.675710,2.816431,-0.564996,-0.460039,-0.631356,...,-2.355290,-1.057265,0.238308,-0.226703,-0.176182,-0.155874,1,1,0,456252
280827,-0.057623,0.222992,0.122570,0.209549,-0.681957,-0.894843,2.503234,-0.564996,-0.460039,-0.081366,...,-2.366949,-0.969384,0.238308,-0.226703,-0.176182,-0.155874,1,1,0,456253
280828,0.016241,-0.563384,0.009711,-0.437270,-0.681957,-0.205802,-0.941940,-0.564996,-0.460039,-0.081366,...,1.284046,0.050775,0.238308,-0.226703,-0.176182,-0.155874,1,1,1,456254


In [7]:
y = pd.read_csv(path+'X_train_target.csv')

In [8]:
X['target'] = y['TARGET']

In [9]:
reference = X.sample(n=5000, replace=False)
current = X.sample(n=5000, replace=False)

In [10]:
#reference['prediction'] = y['TARGET']

## Report

In [11]:
report = Report(metrics=[
    DataDriftPreset(),
])

report.run(reference_data=reference, current_data=current)
report

Output hidden; open in https://colab.research.google.com to view.

In [12]:
report = Report(metrics=[
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile':0.25})
])

report.run(reference_data=reference, current_data=current)
report

Output hidden; open in https://colab.research.google.com to view.

### Generate regular and classification reports

In [13]:
model = joblib.load(path+'my_gbm.pkl')

In [14]:
current_preds = model.predict(current.drop(columns=['SK_ID_CURR', 'target']))

[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.


In [15]:
current['prediction'] = current_preds

In [16]:
ref_preds = model.predict(reference.drop(columns=['SK_ID_CURR', 'target']))
reference['prediction'] = ref_preds

[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.


In [17]:
report = Report(metrics=[
    DataDriftPreset(),
    ClassificationPreset()
])

report.run(reference_data=reference, current_data=current)
report

Output hidden; open in https://colab.research.google.com to view.

## HTML

In [18]:
report.save_html(path+'data_drift_report.html')